# Modelling

### Pipeline
- load the data
- replace null values
- separate categorical and numerical
- remove columns with very high entropy in categorical and numerical
- convert target to binary
- run pca on numerical
- one hot encoding on categorical
- train test separation
- Grid search for dense_nn configuration

### Imports

In [ ]:
%load_ext autoreload

%autoreload 2

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from catboost import CatBoostRegressor, CatBoostClassifier
import pickle

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Lambda
from keras import regularizers
from keras.losses import mean_absolute_error

from keras.wrappers.scikit_learn import KerasClassifier


import math
import numpy as np

from etl.null_value_replacer import NullValueReplacer

from sklearn.base import BaseEstimator, TransformerMixin


### Load Data

In [ ]:
train_data = pd.read_csv("../data/loan-default-prediction/train_v2.csv")

In [ ]:
train_id = train_data["id"]
train_loss = train_data["loss"]

train_data.drop(columns=["id", "loss"], inplace=True)

In [ ]:
train_data, test_data, train_loss, test_loss = train_test_split(train_data, train_loss, test_size=0.33, random_state=42)

In [ ]:
class RedundantColumnsRemover(BaseEstimator, TransformerMixin):
    def fit(self, X, **fit_params):
        df_data_types = X.dtypes
        cat_var = [key for key in dict(df_data_types)
                         if dict(df_data_types)[key] in ['object']]

        tmp = X.drop(columns=cat_var)
        
        column_subset = tmp.columns.values
        groups = []
        redundant_columns = []
        for i in range(len(column_subset)):
            col1 = column_subset[i]
            if col1 in redundant_columns:
                    continue
            same_columns = [col1]

            for j in range(i, len(column_subset)):
                col2 = column_subset[j]
                if col1 == col2:
                    continue
                if (tmp[col1] - tmp[col2]).sum() == 0:
                    same_columns += [col2]
                    redundant_columns += [col2]
            groups+=[same_columns]

        self.columns_to_use = [i[0] for i in groups]
        return self
        
    def transform(self, X, y=None, **transform_params):
        
        return X[self.columns_to_use]
redundant_columns_remover = RedundantColumnsRemover()

In [ ]:
class CategoryEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_encode):
        self.columns_to_encode = columns_to_encode
        
    def fit(self, X, **fit_params):
        return self
        
    def transform(self, X, y=None, **transform_params):
        categorical_pd = X[self.columns_to_encode]
        X_without_categorical = X.drop(columns = self.columns_to_encode)
        categorical_dummies = pd.get_dummies(categorical_pd, columns=self.columns_to_encode)
        return pd.concat([X_without_categorical, categorical_dummies], axis=1)

category_encoder = CategoryEncoder(["f776", "f777", "f725"])

In [ ]:
null_value_replacer = NullValueReplacer("median")
pca = PCA(svd_solver='full')
standard_scaler = StandardScaler()

In [ ]:
train_data = redundant_columns_remover.fit_transform(train_data)
train_data = category_encoder.fit_transform(train_data)
train_data = null_value_replacer.fit_transform(train_data)
# train_data = pca.fit_transform(train_data)
train_data = standard_scaler.fit_transform(train_data)

In [ ]:
train_data = standard_scaler.fit_transform(train_data)

### Linear Regression Classifier

In [ ]:
y = train_loss.astype("bool").astype("int")
X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size=0.33, random_state=42)


In [ ]:
logistic_regression = LogisticRegression()

In [ ]:
logistic_regression.fit(X_train, y_train)

In [ ]:
prediction_logistic = logistic_regression.predict(X_test)

In [ ]:
precision_recall_fscore_support(y_test.values, prediction_logistic)

## Classifier

Do the correlation between target and source columns

In [ ]:
def get_classification_model():
    model=Sequential()

    model.add(Dense(1024, input_dim=X_train.shape[1], kernel_regularizer=regularizers.l1_l2(l1=0.0005, l2=0.0)))
    model.add(Activation("tanh"))
    model.add(Dense(1024, kernel_regularizer=regularizers.l1_l2(l1=0.0005, l2=0.0)))
    model.add(Activation("tanh"))
    model.add(Dense(1))
    model.add(Activation("sigmoid"))



    model.compile(optimizer='adagrad',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
classification_model = get_classification_model()

In [ ]:
classification_model.fit(X_train, y_train, epochs=160, batch_size=4096, validation_data=(X_test, y_test), class_weight={0:1., 1: 10.})

In [ ]:
precision_recall_fscore_support(y_test.values, classification_model.predict_classes(X_test))

In [ ]:
precision_recall_fscore_support(y_train.values, classification_model.predict_classes(X_train))

## Regressor

In [ ]:
foo = pd.DataFrame(data=train_loss.values, columns=["loss"])

In [ ]:
X_r = pd.DataFrame(data=train_data).loc[foo[foo["loss"] > 0].index.values]

In [ ]:
y_r = foo.loc[foo[foo["loss"] > 0].index.values]

In [ ]:
y_r

In [ ]:
X_train_r, X_test_r,y_train_r, y_test_r =  train_test_split(X_r, y_r, test_size=0.33, random_state=42)

In [ ]:
def get_regression_model():
    model=Sequential()

    model.add(Dense(1024, input_dim=X_train_r.shape[1], kernel_regularizer=regularizers.l1_l2(l1=0.00001, l2=0.0)))
    model.add(Dropout(rate=0.2))
    model.add(Activation("tanh"))
    model.add(Dense(1024, kernel_regularizer=regularizers.l1_l2(l1=0.00001, l2=0.0)))
    model.add(Dropout(rate=0.2))
    model.add(Activation("tanh"))
    model.add(Dense(1))

    model.compile(optimizer='adagrad',
                  loss='mean_absolute_error')
    return model

regression_model = get_regression_model()

In [ ]:
regression_model.fit(X_train_r, y_train_r, epochs=200, batch_size=1024, validation_data=(X_test_r, y_test_r))

In [ ]:
regression_prediction = regression_model.predict(X_test_r).reshape(2160,)

In [ ]:
np.round(regression_prediction)[: 20]

In [ ]:
y_test_r.head(20)

In [ ]:
np.abs(regression_prediction - y_test_r["loss"].values).sum()/y_test_r.shape[0]

### Combined prediction

In [ ]:
class CombinedModel():
    def __init__(self,classification_model, regression_model):
        self.classification_model = classification_model
        self.regression_model = regression_model
        
    def predict(self,X):
        classification_prediction = self.classification_model.predict_classes(X)
        indices_of_predicted_defaults = np.where(classification_prediction == 1)[0]
        regression_prediction = self.regression_model.predict(X[indices_of_predicted_defaults])

        result = np.zeros(X.shape[0])
#         np.put(result, indices_of_predicted_defaults, regression_prediction.reshape(regression_prediction.shape[0], ))
        np.put(result, indices_of_predicted_defaults, np.full((regression_prediction.shape[0],), 2) )
        return result

In [ ]:
combined_model = CombinedModel(classification_model, regression_model)


In [ ]:
combined_prediction = combined_model.predict(train_data)

In [ ]:
combined_prediction

In [ ]:
np.abs(combined_prediction - train_loss.values).sum()/combined_prediction.shape[0]


### Submission to Kaggle

In [ ]:
#test_data = pd.read_csv("../data/loan-default-prediction/test_v2.csv")

In [ ]:
#test_ids = test_data["id"]

In [ ]:
#test_data.drop(columns=["id"], inplace=True)

In [ ]:
test_data = redundant_columns_remover.transform(test_data)
test_data = category_encoder.transform(test_data)
test_data = null_value_replacer.transform(test_data)
test_data = pca.transform(test_data)
test_data = standard_scaler.transform(test_data)

In [ ]:
predictions = combined_model.predict(test_data)

In [ ]:
predictions = predictions.reshape(34806,)

In [ ]:
predictions[:30]

In [ ]:
test_loss.values[:30]

In [ ]:
np.abs(predictions - test_loss.values).sum()/predictions.shape[0]


In [ ]:
pd.DataFrame(data={"loss":predictions}, columns=["loss"])

In [ ]:
to_submit = pd.concat([test_ids, pd.DataFrame(data={"loss":predictions}, columns=["loss"])], axis=1)

In [ ]:
to_submit.to_csv("../data/to_submit_3.csv", columns=["id", "loss"], index=False)

In [ ]:
to_submit

### Baselines for predictions

* [ ] do a baseline with mean absolute error against 0
* [ ] do a baseline with mae against existing loss distribution

In [ ]:
train_loss.mean() # This is also a loss against 0

In [ ]:
def calculate_mean_abs_error(predictions, actuals):
    return np.abs(predictions - actuals).sum()/predictions.shape[0]

In [ ]:
calculate_mean_abs_error(np.full(train_loss.shape, 0.8), train_loss) # This is loss against mean value

In [ ]:
random_dist = np.random.choice(
    train_loss.value_counts(normalize=True).sort_index().index.values,
    train_loss.shape[0],
    p=train_loss.value_counts(normalize=True).sort_index().values)

In [ ]:
calculate_mean_abs_error(random_dist, train_loss) # This is loss for random distribution with same prob values

In [ ]:
train_loss[train_loss> 0].mean()